In [ ]:
import pandas as pd

df = pd.read_csv("Fish.csv", sep=",", decimal=".")
sp = {
    "Perch": 0, "Bream": 1, "Roach": 2, "Pike": 3, "Smelt": 4, "Parkki": 5, "Whitefish": 6}

df["Species"] = df["Species"].map(sp)
df = df.sample(frac=1, random_state=42).reset_index(drop=True)
df


,Species,Weight,Length1,Length2,Length3,Height,Width
0,0,78.0,16.8,18.7,19.4,5.1992,3.1234
1,4,13.4,11.7,12.4,13.5,2.4300,1.2690
2,3,200.0,30.0,32.3,34.8,5.5680,3.3756
3,6,270.0,23.6,26.0,28.7,8.3804,4.2476
4,0,150.0,21.0,23.0,24.5,5.2185,3.6260
...,...,...,...,...,...,...,...
154,5,300.0,24.0,26.0,29.0,11.3680,4.2340
155,0,250.0,25.9,28.0,29.4,7.8204,4.2042
156,1,600.0,29.4,32.0,37.2,14.9544,5.1708
157,0,150.0,20.5,22.5,24.0,6.7920,3.6240


In [2]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

X = df.drop(columns="Weight")
y = df["Weight"]

scaler = MinMaxScaler(feature_range=(0,1))

X = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [27]:
from sklearn.ensemble import RandomForestRegressor, BaggingRegressor
from xgboost import XGBRegressor

model_rnd_forest = RandomForestRegressor(n_estimators=100, random_state=42, max_depth=7)
model_rnd_forest.fit(X_train, y_train)
model_rnd_forest.feature_importances_

array([0.00125681, 0.05626106, 0.04508157, 0.21595974, 0.01355232,
       0.6678885 ])

In [ ]:
model_bagging = BaggingRegressor(n_estimators=50, random_state=42, max_samples=0.8)
model_bagging.fit(X_train, y_train)

,estimator,None
,n_estimators,50
,max_samples,0.8
,max_features,1.0
,bootstrap,True
,bootstrap_features,False
,oob_score,False
,warm_start,False
,n_jobs,None
,random_state,42
,verbose,0


In [31]:
model_boost = XGBRegressor(n_estimators=100, learning_rate=0.5)
model_boost.fit(X_train, y_train)
model_boost.feature_importances_

array([1.8483821e-04, 3.5770476e-02, 1.2906388e-02, 2.8769106e-01,
       8.2747890e-03, 6.5517241e-01], dtype=float32)

In [32]:
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error, r2_score

models = [model_rnd_forest, model_bagging, model_boost]

for model in models:
    y_pred = model.predict(X_test)
    
    test_mse = mean_squared_error(y_test, y_pred)
    test_r2 = r2_score(y_test, y_pred)
    test_mape = mean_absolute_percentage_error(y_test, y_pred)

    print(model)
    print(f"mse: {test_mse}\nr2: {test_r2}\nmape: {test_mape}\n\n")  


RandomForestRegressor(max_depth=7, random_state=42)
mse: 3840.5839273735446
r2: 0.9726963296151604
mape: 0.08493610100410953


BaggingRegressor(max_samples=0.8, n_estimators=50, random_state=42)
mse: 4298.5328258
r2: 0.9694406565164378
mape: 0.09290490001652703


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             feature_weights=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=0.5, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=100,
             n_jobs=None, num_parallel_tree

- <font color="purple">MSE (Mean Squared Error)</font> — среднеквадратичная ошибка. Измеряет среднее квадратов разниц между предсказанными и реальными значениями. Чем меньше — тем лучше. Штрафует сильнее за большие ошибки. Значения ~3700–4200 — так как в датасете очень большой разброс и есть данные с околонулевым значением, это приемлемо.
- <font color="purple"> R² (коэффициент детерминации)</font> — доля дисперсии целевой переменной, объяснённая моделью. От 0 до 1 (может быть отрицательным при плохой модели). Чем ближе к 1 — тем лучше. ~0.97–0.973 — отличный результат, модель объясняет более 97% вариации данных.
- <font color="purple">MAPE (Mean Absolute Percentage Error)</font>  — средняя абсолютная процентная ошибка. Показывает, на сколько процентов в среднем предсказания отклоняются от реальных значений. Чем меньше — тем лучше. Обычно:
    * 0-10% — очень хорошо,
    * 10–20% — приемлемо,
    * +20% — слабо.